In [1]:
import json
import requests
import utils
import threading
from playsound import playsound
from configs import API_KEY,BASE_URL,AMOUNT,MIN_PERCENT_REQUIRED
from biscoint import get_rate_limit,get_balance,get_offer,confirm_offer
import time

In [4]:
#Inicial configs
rate_limit = get_rate_limit()
sleep_time_offers = ((rate_limit["windowMs"] / rate_limit["maxRequests"]) / 1000)*2
initial_balance = get_balance()
last_balance = initial_balance

In [7]:
%%time
buy = get_offer('buy','0.0001',False)
sell = get_offer('sell','0.0001',False)
print(sell)

{'offerId': 'cFWvDLznKYWg7FfYs', 'base': 'BTC', 'quote': 'BRL', 'op': 'sell', 'isQuote': False, 'baseAmount': '0.00010000', 'quoteAmount': '32.29', 'efPrice': '322900.00', 'createdAt': '2021-04-19T01:32:38.470Z', 'expiresAt': '2021-04-19T01:32:53.470Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}
Wall time: 2.03 s


In [3]:
def async_offer(op:str,amount:str,isQuote:bool):
    try:
        print(op,'started')
        response = get_offer(op,amount,isQuote)
        request_orders[op] = response
    except Exception as e:
        print("Erro ao rodar thread",e)

In [8]:
%%time
try:
    request_orders = {'buy':None,'sell':None}

    thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
    thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
    thread_buy.start()
    time.sleep(0.1)
    thread_sell.start()
    thread_buy.join()
    thread_sell.join()

    buy_price = request_orders['buy']['efPrice']
    sell_price = request_orders['sell']['efPrice']
    percent = utils.percent(buy_price, sell_price)

    if percent >= MIN_PERCENT_REQUIRED:
        print(f"Arbitrage oportunity: buy:{buy_price}   sell:{sell_price} ->  {percent}%")
        playsound('beep.wav')


    print(percent)

    print('finished')
except Exception as e:
    print("Erro")
    print(e)



buy started
sell started
-0.92
finished
Wall time: 2.41 s


In [9]:
request_orders

{'buy': None,
 'sell': {'offerId': 'GCaAqscFuLjREEhJ8',
  'base': 'BTC',
  'quote': 'BRL',
  'op': 'sell',
  'isQuote': False,
  'baseAmount': '0.00001000',
  'quoteAmount': '3.20',
  'efPrice': '320000.00',
  'createdAt': '2021-04-19T00:09:55.529Z',
  'expiresAt': '2021-04-19T00:10:10.529Z',
  'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}}

In [9]:
#confirm_offer('EAWBrEhsrAtbe8BXE')

{'code': 5113,
 'error': 'Offer amount is off limits',
 'details': '฿\xa00.00010000 <= offer <= ฿\xa01.00000000'}